# Exercises for 2021-05-13

In this jupyter notebook, we are taking a look at an example for data-driven Koopman with the lifted DMD. Note that you will need to import the "DifferentialEquation" package and the "FFTW" package if these are not yet installed. 

In [ ]:
using Pkg
Pkg.add("DifferentialEquations")
Pkg.add("FFTW")

In [ ]:
using Plots
using Random
using LinearAlgebra
using DifferentialEquations
using FFTW

# Data-driven Koopman 

In the lecture, we had examples for which we 'easily' could find a solution or we looked at cases that can be solve via a Taylor or Laurent series expansion. Here, we will look at a more realistic example, the nonlinear Schrödinger equation:
$$
i \frac{\partial q}{\partial t} + \frac{1}{2}\frac{\partial^2 q}{\partial \xi^2}+ |q|^2 q =0,
$$
with $q(\xi,t)$ being a function in space and time.
The first couple of code snippets will generate the data we use for the DMD. How the data is generated is not the main focus of the notebook - but we nevertheless need data to work with for our data-driven algorithms. (But in case you're interested feel free to read about the nonlinear Schrödinger equation [here](https://en.wikipedia.org/wiki/Nonlinear_Schrödinger_equation)).

In [ ]:
#Defining our space domain
L=30; 
n=128;
x1 = range(-L/2, stop=L/2, length= n+1)
#state space
xi=x1[1:n];

#Wave numbers 
k=(2*pi/L)*[0:n/2-1;-n/2:-1];

Next, we define our time discretization. 

In [ ]:
slices = 128;
t1 = range(0, stop=pi , length = slices+1); 
deltat1 = t1[2]-t1[1];

The above equation can be rewritten and is then transformed via a Fourier transormation in space. We solve the ODE problem here to generate data for our DMD and Koopman DMD. 

In [ ]:
# define right-hand-side
function dmd_soliton_rhs(dy,y,t,p) 
    q = ifft(y);
    dy[:] = -(1im/2)*(k .^2).*y[:]+1im*fft((broadcast(abs,q).^2).*q);    
end

# Initial condition
ic(x) = 2* sech(x);
Initial_data = [ic(x) for x in xi]

qt= (fft(Initial_data));

tspan = (0.,pi)
prob1 = ODEProblem(dmd_soliton_rhs, qt ,tspan)
sol = solve(prob1, Vern7(), adaptive=false ,dt = deltat1);

Let's take a look how our solution looks like! This will be the data we are working with on this notebook.

In [ ]:
qsol = Matrix{ComplexF64}(undef, n, 0)
for iter in range(1,stop=slices+1)
    qsol = hcat(qsol,ifft(sol.u[iter]) )
end

p1= surface(xi,t1,broadcast(abs,qsol)') 
#p1 = heatmap(xi,t1,broadcast(abs,qsol)')
plot(p1)

## Performing usual DMD as comparison

We start by a state-space DMD to see how good we perform with states as measurements only. 

In [ ]:
X = qsol;

We create the DMD matrices by
$$
X_1= [x_0, \ldots, x_{K-1} ], \quad X_2= [x_1, \ldots, x_{K} ]
$$
such that we seek
$$
A = X_2 X_1^{\dagger},
$$
where $\cdot^{\dagger}$ denotes the Moore-Penrose pseudoinverse.

In [ ]:
#create DMD data matrices
X1 = X[:, 1:end-1];
X2 = X[:, 2:end];

Next, we compute the SVD of $X_1$ in order to subsequently compute the pseudoinverse:
$$
X_1 = U \Sigma V^{\ast},
$$
and truncate the matrices. We arrive at the approximation
$$
X_1 \approx \tilde{U} \tilde{\Sigma} \tilde{V}^{\ast}.
$$

In [ ]:
#SVD and rank truncation - here rank 10
r=10
SVDX = svd(X1)

#truncate matrices
Utilde = SVDX.U[:,1:r];
S = Diagonal(SVDX.S);
Stilde = S[1:r,1:r];
Vtilde = SVDX.V[:,1:r];

Next, we compute 
$$
\tilde{A} = \tilde{U}^{*} X_2 \tilde{V} \tilde{\Sigma}^{-1}, 
$$ 
perform the eigendecomposition of $\tilde{A}$ and compute the dynamic modes $\phi$ of $\bar{A}.$

In [ ]:
Atilde = Utilde'*X2*Vtilde/Stilde;
Lambda = Diagonal(eigvals(Atilde));
W = eigvecs(Atilde)
omega = log(Lambda)/deltat1;
Phi = X2*Vtilde/Stilde*W; # DMD Modes

Compute the DMD solution via 
$$
x(t)= \Phi \exp(\Omega t)b,
$$
where b are the coordinates of the initial condition in eigenvector basis.

In [ ]:
#compute dmd solution
x_1 = X[:, 1];
b = Phi\x_1; 

time_dynamics = Matrix(undef, r, 0)
for iter in range(1,stop=slices+1)
    time_dynamics = hcat(time_dynamics,exp(omega*t1[iter])*b) 
end 

X_dmd = Phi*time_dynamics;

We take a look at the DMD solution.

In [ ]:
p2 = surface(xi,t1,broadcast(abs,X_dmd)') 
#p2 = heatmap(xi,t1,broadcast(abs,X_dmd)')
plot(p2)

## DMD lifted to observable space
Can we do better by including meaurement functions? What would be a good guess on your side to include as a measurement function? For now we keep the state as out first measurement function, but you can add a second measurement.

We create the DMD matrices (here called $Y_1$ and $Y_2$) by
$$
Y_1= [g(x_0), \ldots, g(x_{K-1}) ], \quad Y_2= [g(x_1), \ldots, g(x_{K}) ]
$$
such that we seek
$$
A_Y = Y_2 Y_1^{\dagger}.
$$

In [ ]:
Y1=[X1; ?insert measurement here for X1?]; 
Y2=[X2; ?insert same measurement here for X2?];

Here, we also compute the SVD, but of the lifted matrix $Y_1$:
$$
Y_1 = U_Y \Sigma_Y V_Y^{\ast},
$$
and truncate the matrices. We arrive at the approximation
$$
Y_1 \approx \tilde{U}_Y \tilde{\Sigma}_Y \tilde{V}_Y^{\ast}.
$$

In [ ]:
#SVD and rank truncation - here rank 10
rY=10
SVDY = svd(Y1)

#truncate matrices
UtildeY = SVDY.U[:,1:rY];
SY = Diagonal(SVDY.S);
StildeY = S[1:rY,1:rY];
VtildeY = SVDY.V[:,1:rY];

We compute 
$$
\tilde{A}_Y = \tilde{U}_Y^{*} Y_2 \tilde{V}_Y \tilde{\Sigma}_Y^{-1}, 
$$ 
perform the eigendecomposition of $\tilde{A}_Y$ and compute the dynamic modes $\phi_Y$ of $\bar{A}_Y.$

In [ ]:
AtildeY = UtildeY'*Y2*VtildeY/StildeY;
LambdaY = Diagonal(eigvals(AtildeY));
WY = eigvecs(AtildeY)
omegaY = log(LambdaY)/deltat1;
PhiY = Y2*VtildeY/StildeY*WY; # DMD Modes

Compute the DMD solution via 
$$
y(t)= \Phi_Y \exp(\Omega_Y t)b_Y,
$$
where b_Y are the coordinates of the initial condition in eigenvector basis.

In [ ]:
#compute extended dmd solution
q2 = [Initial_data; ?insert initial measurement here?]
y0 = PhiY\q2; 

time_dynamicsY = Matrix(undef, rY, 0)
for iter in range(1,stop=slices+1)
    time_dynamicsY = hcat(time_dynamicsY,exp(omegaY*t1[iter])*y0) 
end 

Y_dmd2 = PhiY*time_dynamicsY;

How do we actually transform back our state measurements in this case?

In [ ]:
#Koopman approximation
q_dmd = ?

In [ ]:
p3 = surface(xi,t1,broadcast(abs,q_dmd)') 
#p3 = heatmap(xi,t1,broadcast(abs,q_dmd)')
plot(p3)